In [3]:
import pandas as pd
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url = '/workspaces/metro-JO/Données importées/2022_S2_PROFIL_FER.csv'
données_validation = pd.read_csv(url, delimiter=';')
display(données_validation)

/tmp/ipykernel_4701/676259131.py:4: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  données_validation = pd.read_csv(url, delimiter=';')


,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_ARRET,LIBELLE_ARRET,lda,CAT_JOUR,TRNC_HORR_60,pourc_validations
0,100,110,1,PORTE MAILLOT,71379,DIJFP,0H-1H,"2,18"
1,100,110,1,PORTE MAILLOT,71379,DIJFP,10H-11H,"5,03"
2,100,110,1,PORTE MAILLOT,71379,DIJFP,11H-12H,"5,72"
3,100,110,1,PORTE MAILLOT,71379,DIJFP,12H-13H,"6,10"
4,100,110,1,PORTE MAILLOT,71379,DIJFP,13H-14H,"5,87"
...,...,...,...,...,...,...,...,...
85249,810,802,905,VILLEPINTE,73547,SAVS,5H-6H,"4,25"
85250,810,802,905,VILLEPINTE,73547,SAVS,6H-7H,"7,22"
85251,810,802,905,VILLEPINTE,73547,SAVS,7H-8H,"6,79"
85252,810,802,905,VILLEPINTE,73547,SAVS,8H-9H,"6,39"


In [4]:
# Convertir la colonne 'pourc_validations' en float (remplacer la virgule par un point pour séparer les décimales)
données_validation['pourc_validations'] = données_validation['pourc_validations'].str.replace(',', '.').astype(float)

# Grouper par 'LIBELLE_ARRET' et 'TRNC_HORR_60', puis calculer la somme des pourc_validations pour chaque groupe
tableau_somme_pour_validations = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()

# Afficher le tableau obtenu
print("Tableau avec la somme des pourc_validations pour chaque arrêt et chaque tranche horaire :")
print(tableau_somme_pour_validations)


Tableau avec la somme des pourc_validations pour chaque arrêt et chaque tranche horaire :
        LIBELLE_ARRET TRNC_HORR_60  pourc_validations
0      ABBESSES              0H-1H              10.18
1      ABBESSES            10H-11H              19.26
2      ABBESSES            11H-12H              23.72
3      ABBESSES            12H-13H              27.00
4      ABBESSES            13H-14H              29.13
...               ...          ...                ...
17610  YVRIS NOISY GD        5H-6H              10.08
17611  YVRIS NOISY GD        6H-7H              36.43
17612  YVRIS NOISY GD        7H-8H              69.33
17613  YVRIS NOISY GD        8H-9H              73.34
17614  YVRIS NOISY GD       9H-10H              35.38

[17615 rows x 3 columns]


In [5]:
# Mapping des tranches horaires à des heures spécifiques
tranche_horaire_to_hour = {
    '0H-1H': '00:00-01:00',
    '1H-2H': '01:00-02:00',
    '2H-3H': '02:00-03:00',
    '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00',
    '5H-6H': '05:00-06:00',
    '6H-7H': '06:00-07:00',
    '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00',
    '9H-10H': '09:00-10:00',
    '10H-11H': '10:00-11:00',
    '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00',
    '13H-14H': '13:00-14:00',
    '14H-15H': '14:00-15:00',
    '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00',
    '17H-18H': '17:00-18:00',
    '18H-19H': '18:00-19:00',
    '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00',
    '21H-22H': '21:00-22:00',
    '22H-23H': '22:00-23:00',
    '23H-24H': '23:00-00:00'
}

# Appliquer la conversion des tranches horaires au format heure
tableau_somme_pour_validations['TRNC_HORR_60'] = tableau_somme_pour_validations['TRNC_HORR_60'].map(tranche_horaire_to_hour)

# Afficher le tableau avec les tranches horaires converties en heures
print("Tableau avec les tranches horaires converties au format heure :")
print(tableau_somme_pour_validations)


Tableau avec les tranches horaires converties au format heure :
        LIBELLE_ARRET TRNC_HORR_60  pourc_validations
0      ABBESSES        00:00-01:00              10.18
1      ABBESSES        10:00-11:00              19.26
2      ABBESSES        11:00-12:00              23.72
3      ABBESSES        12:00-13:00              27.00
4      ABBESSES        13:00-14:00              29.13
...               ...          ...                ...
17610  YVRIS NOISY GD  05:00-06:00              10.08
17611  YVRIS NOISY GD  06:00-07:00              36.43
17612  YVRIS NOISY GD  07:00-08:00              69.33
17613  YVRIS NOISY GD  08:00-09:00              73.34
17614  YVRIS NOISY GD  09:00-10:00              35.38

[17615 rows x 3 columns]


In [6]:
# Regroupement des données en cas de doublons pour une paire 'LIBELLE_ARRET'-'TRNC_HORR_60'
grouped_data = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()

# Pivotement des données
pivot_table = grouped_data.pivot(index='LIBELLE_ARRET', columns='TRNC_HORR_60', values='pourc_validations').reset_index()

# Renommage des colonnes
pivot_table.columns.name = None  # Supprimer le nom de la colonne index
pivot_table = pivot_table.rename(columns=lambda x: f"{x}" if x != 'LIBELLE_ARRET' else x)

# Affichage du tableau pivoté
print("Tableau pivoté avec les pourcentages de validations pour chaque tranche horaire par LIBELLE_ARRET :")
print(pivot_table)



Tableau pivoté avec les pourcentages de validations pour chaque tranche horaire par LIBELLE_ARRET :
       LIBELLE_ARRET  0H-1H  10H-11H  11H-12H  12H-13H  13H-14H  14H-15H  \
0     ABBESSES        10.18    19.26    23.72    27.00    29.13    34.04   
1              ABLON   0.26    27.97    28.99    27.78    27.78    27.21   
2      ACHERES GRAND   0.78    22.18    32.89    29.44    32.44    29.83   
3      ACHERES VILLE   0.76    30.57    30.81    31.27    30.99    31.10   
4      AEROPORT CDG1   0.76    19.15    21.41    28.14    31.14    39.21   
..               ...    ...      ...      ...      ...      ...      ...   
751           VOSVES    NaN    12.17    16.96    26.13    20.53    46.84   
752  VULAINES SUR SE   0.11    37.30    27.07    24.99    36.22    34.67   
753   WAGRAM           3.63    25.53    29.78    35.14    34.21    34.50   
754           YERRES   0.73    30.77    31.42    28.88    27.55    27.72   
755   YVRIS NOISY GD   0.20    26.26    26.39    26.45    25.95 

In [7]:
import re

# Sélectionner les colonnes correspondant aux tranches horaires
horaire_columns = pivot_table.columns[1:]  # Exclure la colonne 'LIBELLE_ARRET'

# Fonction pour extraire l'heure de début de chaque tranche horaire
def extract_start_hour(column):
    match = re.match(r'(\d+)H-(\d+)H', column)
    if match:
        return int(match.group(1))
    return 0

# Trier les colonnes dans l'ordre croissant des heures de début des tranches
sorted_columns = sorted(horaire_columns, key=extract_start_hour)

# Réorganiser le tableau pivoté avec les colonnes dans l'ordre chronologique
new_column_order = ['LIBELLE_ARRET'] + sorted_columns
pivot_table = pivot_table[new_column_order]

In [8]:
# Supprimer la colonne 'ND' du DataFrame pivot_table_ordered
pivot_table.drop(columns=['ND'], inplace=True, errors='ignore')

pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.strip()  # Supprime les espaces en début et fin
pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.lower()  # Met tout en minuscules

# Créer un dictionnaire de correspondance pour renommer les colonnes
correspondance_heures = {
    '0H-1H': '00:00-01:00', '1H-2H': '01:00-02:00', '2H-3H': '02:00-03:00', '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00', '5H-6H': '05:00-06:00', '6H-7H': '06:00-07:00', '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00', '9H-10H': '09:00-10:00', '10H-11H': '10:00-11:00', '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00', '13H-14H': '13:00-14:00', '14H-15H': '14:00-15:00', '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00', '17H-18H': '17:00-18:00', '18H-19H': '18:00-19:00', '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00', '21H-22H': '21:00-22:00', '22H-23H': '22:00-23:00', '23H-0H': '23:00-00:00'
}

# Renommer les colonnes en utilisant le dictionnaire de correspondance
pivot_table.rename(columns=correspondance_heures, inplace=True)

# Afficher le DataFrame après suppression de la colonne 'ND'
print("Tableau pivoté après suppression de la colonne 'ND' :")
print(pivot_table)

Tableau pivoté après suppression de la colonne 'ND' :
       LIBELLE_ARRET  00:00-01:00  01:00-02:00  02:00-03:00  03:00-04:00  \
0           abbesses        10.18         3.20         0.02         0.00   
1              ablon         0.26         0.03         0.00         0.02   
2      acheres grand         0.78          NaN          NaN          NaN   
3      acheres ville         0.76         0.31         0.13         0.09   
4      aeroport cdg1         0.76         0.14         0.01         0.04   
..               ...          ...          ...          ...          ...   
751           vosves          NaN          NaN          NaN          NaN   
752  vulaines sur se         0.11          NaN          NaN          NaN   
753           wagram         3.63         1.43         0.10         0.00   
754           yerres         0.73         0.11         0.01         0.05   
755   yvris noisy gd         0.20         0.04          NaN          NaN   

     04:00-05:00  05:00-06:00  06

In [9]:
#Nettoyage de données pour les validations par station et horaires: pour chaque station, on a le pourcentage d'affluence à chaque horaire sur une journée type
import pandas as pd
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url = 'http://opendata.stif.info/explore/dataset/validations-reseau-ferre-profils-horaires-par-jour-type-1er-semestre/download?format=csv&timezone=Europe/Berlin&use_labels_for_header=false'
données_validation = pd.read_csv(url, delimiter=';')
données_validation_station_horaire = données_validation[['libelle_arret', 'trnc_horr_60','pourc_validations']]
donnees_organisees = données_validation_station_horaire.pivot_table(index='libelle_arret', columns='trnc_horr_60', values='pourc_validations', aggfunc='mean').reset_index()
print(donnees_organisees)

trnc_horr_60    libelle_arret   0H-1H  10H-11H  11H-12H  12H-13H  13H-14H  \
0                    ABBESSES  1.9100    3.792    4.776    5.672    6.022   
1                       ABLON  0.0375    5.314    5.110    5.018    4.816   
2               ACHERES GRAND  0.1200    4.506    5.692    5.470    6.676   
3               ACHERES VILLE  0.2100    5.934    6.148    6.384    6.170   
4               AEROPORT CDG1  0.1260    4.102    4.486    5.208    6.560   
..                        ...     ...      ...      ...      ...      ...   
742                    VOSVES     NaN    4.360    4.218    9.368    4.512   
743           VULAINES SUR SE     NaN    7.312    4.808    5.806    6.052   
744                    WAGRAM  0.7640    5.088    5.930    6.988    6.962   
745                    YERRES  0.1200    6.058    6.192    5.880    5.788   
746            YVRIS NOISY GD  0.0000    4.936    5.674    5.322    5.284   

trnc_horr_60  14H-15H  15H-16H  16H-17H  17H-18H  ...    23H-0H     2H-3H  

In [10]:
#Nettoyage de données sur le nombre de validations: on obtient le nombre de validation moyen par jour sur 1 mois précis

%config NotebookApp.iopub_data_rate_limit=100000000000.0
url2 = '/workspaces/metro-JO/Données importées/2022_S2_NB_FER.csv'
données_validation2= pd.read_csv(url2)
données_validation2 = pd.read_csv(url2, delimiter=';')
print(données_validation2)

# Exemple avec gestion de formats flexibles
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], errors='coerce', format="%d/%m/%Y")
donnees_mai = données_validation2.loc[(données_validation2['JOUR'].dt.month == 5)]
donnees_sum = donnees_mai.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Conversion de la colonne 'JOUR' en datetime
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrer les données pour les dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Grouper par 'JOUR', 'LIBELLE_ARRET' et 'CATEGORIE_TITRE' et calculer la somme des 'NB_VALD'
donnees_aggreg = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET', 'CATEGORIE_TITRE'])['NB_VALD'].sum().reset_index()

# Conversion de la colonne 'JOUR' en datetime si ce n'est pas déjà fait
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrage des dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Somme de NB_VALD par JOUR et par LIBELLE_ARRET
somme_par_jour_arret = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET'])['NB_VALD'].sum().reset_index()

moyenne_par_arret = données_validation2.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

               JOUR  CODE_STIF_TRNS CODE_STIF_RES CODE_STIF_ARRET  \
0        01/07/2022             100           110               1   
1        01/07/2022             100           110               1   
2        01/07/2022             100           110               1   
3        01/07/2022             100           110               1   
4        01/07/2022             100           110               1   
...             ...             ...           ...             ...   
1105942  31/12/2022             810           802             905   
1105943  31/12/2022             810           802             905   
1105944  31/12/2022             810           802             905   
1105945  31/12/2022             810           802             905   
1105946  31/12/2022             810           802             905   

          LIBELLE_ARRET    lda CATEGORIE_TITRE  NB_VALD  
0        PORTE MAILLOT   71379               ?      253  
1        PORTE MAILLOT   71379       AMETHYSTE      171